<a href="https://colab.research.google.com/github/Crane-YU/tensorflow_project/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib as mpl
mpl.use("Agg")
from matplotlib import pyplot as plt


In [0]:
HIDDEN_SIZE = 30
NUM_LAYERS = 2

TIME_STEPS = 10
TRAINING_STEPS = 10000
BATCH_SIZE = 32

TRAINING_EXAMPLES = 10000
TESTING_EXAMPLES = 1000
SAMPLE_GAP = 0.01


In [0]:
def generate_data(seq):
  X = []
  y = []
  
  for i in range(len(seq) - TIME_STEPS):
    X.append([seq[i:i + TIME_STEPS]])
    y.append([seq[i + TIME_STEPS]])
  
  return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

In [0]:
def lstm_model(X, y, is_training):
  cells = [tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) for _ in range(NUM_LAYERS)]
  stacked_rnn_cell  = tf.nn.rnn_cell.MultiRNNCell(cells)
  
  outputs, _ = tf.nn.dynamic_rnn(stacked_rnn_cell, X, dtype=np.float32, time_major=False)
  output = outputs[:, -1, :]
  
  predictions = tf.contrib.layers.fully_connected(output, 1, activation_fn=None)
  
  if not is_training:
    return predictions, None, None
  
  loss = tf.losses.mean_squared_error(labels=y, predictions=predictions)
  
  train_op = tf.contrib.layers.optimize_loss(
      loss, tf.train.get_global_step(), optimizer="Adagrad", learning_rate=0.1
  )
  
  return predictions, loss, train_op

In [0]:
def train(sess, train_X, train_Y):
  ds = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
  ds = ds.repeat().shuffle(1000).batch(BATCH_SIZE)
  X, y = ds.make_one_shot_iterator().get_next()
  
  with tf.variable_scope("model"):
    predictions, loss, train_op = lstm_model(X, y, True)
  
  sess.run(tf.global_variables_initializer())
  
  for i in range(TRAINING_STEPS):
    _, loss = sess.run([train_op, loss])
    if i % 100 == 0:
      print("Train step: " + str(i) + ", loss: " + str(loss))

In [0]:
def run_eval(sess, test_X, test_Y):
  ds = tf.data.Dataset.from_tensor_slices((test_X, test_Y))
  ds = ds.batch(1)
  X, y = ds.make_one_shot_iterator().get_next()
  
  with tf.variable_scope("model", reuse=True):
    prediction, _, _ = lstm_model(X, [0.0], False)
  
  predictions = []
  labels = []
  
  for i in range(TRAINING_STEPS):
    p, l = sess.run([prediction, y])
    predictions.append(p)
    labels.append(l)
  
  predictions = np.array(predictions).squeeze()
  labels = np.array(labels).squeeze()
  rmse = np.sqrt(((predictions - labels)**2).mean(axis=0))
  
  plt.figure()
  plt.plot(predictions, label="prediction")
  plt.plot(labels, label="real_sin")
  plt.legend()
  plt.show()

In [45]:
test_start = (TRAINING_EXAMPLES + TIME_STEPS)*SAMPLE_GAP
test_end = test_start + (TESTING_EXAMPLES + TIME_STEPS)*SAMPLE_GAP

train_X, train_Y = generate_data(np.sin(np.linspace(0, test_start, TRAINING_EXAMPLES+TIME_STEPS, dtype=np.float32)))
test_X, test_Y = generate_data(np.sin(np.linspace(test_start, test_end, TESTING_EXAMPLES+TIME_STEPS, dtype=np.float32)))

with tf.Session() as sess:
  train(sess, train_X, train_Y)
  run_eval(sess, test_X, test_Y)

ValueError: ignored